In [1]:
# library imports
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
# data import
df = pd.read_csv("C:\\Users\\RoiMinuit\\Desktop\\data\\finance\\synthetic_credit_risk_data_historical.csv")
print(df.iloc[:5, :5])

   Customer_ID  Age Employment_Type  Income_Level  Avg_Monthly_Deposits
0            1   32      Freelancer        165216                  8396
1            2   49        Salaried        149181                  2778
2            3   41   Self-Employed         34471                  6045
3            4   31      Freelancer         37301                  2352
4            5   57        Salaried         98318                  6705


In [8]:
# data cleaning
df.drop(columns=['Customer_ID'], inplace=True)
df.iloc[:5, :5]

,Age,Employment_Type,Income_Level,Avg_Monthly_Deposits,Avg_Monthly_Withdrawals
0,32,Freelancer,165216,8396,9844
1,49,Salaried,149181,2778,8939
2,41,Self-Employed,34471,6045,1445
3,31,Freelancer,37301,2352,6971
4,57,Salaried,98318,6705,8644


In [ ]:
# features for training
X = df.drop(columns=['Loan_Approval_Status'])
y = df['Loan_Approval_Status']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5)